In [93]:
# importing the required libraries 
import pandas as pd
import numpy as np

# Reading a folder/directory
import os
import glob

pd.set_option('display.max_columns', None)

In [94]:
# Define the directory where your files are located
directory = r'data\plant3D'

# Define the pattern to match Excel files with epoch time suffix (e.g., '_<epoch_time>_latest')
pattern = '*.csv'

# Get a list of CSV files that match the pattern in the directory
if os.listdir(directory):
    csv_files = glob.glob(os.path.join(directory, pattern))

    # Check if any csv files were found
    if csv_files:
        print("Found CSV Files:")
        for item in csv_files:
            print(item)
    else:
        print("No CSV files found in the directory")

else:
    print("Directory is Empty")


Found CSV Files:
data\plant3D\Iso_PnPDrawings_PNP.csv
data\plant3D\merged_output.csv
data\plant3D\Ortho_PnPDrawings_PNP.csv
data\plant3D\Piping_Pipe_PNP.csv


In [95]:
df_iso = pd.read_csv(csv_files[0], low_memory=False)
df_ortho = pd.read_csv(csv_files[2], low_memory=False)
df_pipingPnP = pd.read_csv(csv_files[3], low_memory=False)

print(f'Data in Iso file {df_iso.shape}')
print(f'Data in Ortho file {df_ortho.shape}')
print(f'Data in PipingPnP file {df_pipingPnP.shape}')

Data in Iso file (42, 28)
Data in Ortho file (15, 30)
Data in PipingPnP file (6597, 80)


### Variables Calculations

#### Piping LF

In [96]:
## Estimated
"""
This value will directly come from estimation data 
"""
pipe_lf_estimated = 1500 # Fixed value

## Modeled
"""
Sum of length from Pipe_Pnp database (Cut Length/304.8) (cut length/length both can be used)
"""
pipe_lf_modeled = np.round(np.divide(df_pipingPnP['CutLength'].agg('sum'),304.8),2)

#### Line Numbered

In [97]:
## Estimated 
""""
Sum of CutLength column from Pipe PnP database 
"""
line_numbered_estimated = np.round(df_pipingPnP['CutLength'].agg('sum'),2)

## Modeled
""" 
Sum of CutLength from Pipe Pnp database where the LinerNumberTag column does not contain “?”.
"""
line_numbered_modeled = np.round(df_pipingPnP[~df_pipingPnP['LineNumberTag'].fillna("").str.contains("\\?")]['CutLength'].agg('sum'),2)

In [98]:
# df_pipingPnP.CutLength.dtypes

#### Spool Maps

In [99]:
## Estimated 
"""
Sum of cut length from Pipe Pnp database divided by 60*308.4
"""
spool_maps_estimated = np.round(np.divide(df_pipingPnP['CutLength'].agg('sum'),(60*304.8)),2)

## Modeled
"""
Logic not clear yet
"""
spool_maps_modeled = None

#### ISO/Spool

In [100]:
## Estimated
"""
Unique count of Dwg Name column from ISO database and PnPDrawing_PNP table
"""
iso_spool_estimeted = df_iso['Dwg Name'].nunique()

## Modeled
"""
Sum of cut length from Pnp database where SpoolNumber column is not null
"""
iso_spool_modeled = np.round(df_pipingPnP[~df_pipingPnP['SpoolNumber'].isnull()]['CutLength'].agg('sum'),2)

#### Ortho

In [101]:
## Estimated
"""
Directly from Estimation data
"""
ortho_count_estimated = None

## Modeled
"""
Unique count of Dwg Name column from Ortho database and PnPDrawing_PNP table
"""
ortho_count_modeled = df_ortho['Dwg Name'].nunique()

### Creating a DataFrame to hold these values

In [102]:
data = {
    'Category' : ['Pipe LF', 'Line Numbered', 'Spool Maps', 'Iso/Spool', 'Ortho'],
    'Estimated' : [pipe_lf_estimated, line_numbered_estimated, spool_maps_estimated, iso_spool_estimeted, ortho_count_estimated],
    'Modeled' : [pipe_lf_modeled, line_numbered_modeled, spool_maps_modeled, iso_spool_modeled, ortho_count_modeled]
}

In [103]:
final_df = pd.DataFrame(data)
final_df

,Category,Estimated,Modeled
0,Pipe LF,1500.00,1242.99
1,Line Numbered,378863.47,364297.98
2,Spool Maps,20.72,NaN
3,Iso/Spool,42.00,248642.76
4,Ortho,NaN,15.00
